You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [1]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [2]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [3]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [4]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [5]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [6]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [7]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [8]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit], # tasks operates sequentially i.e first it will plan then write and then edit
    verbose=2
)

2024-11-27 07:02:50,931 - 140515715181440 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

**Content Plan: Artificial Intelligence**

**Introduction:**
Artificial Intelligence (AI) has become a game-changer in various industries, revolutionizing the way businesses operate and improving efficiency. In this blog article, we will explore the latest trends, key players, and noteworthy news in the field of AI.

**Key Points:**

1. **Latest Trends in Artificial Intelligence:**
   - Machine learning algorithms continue to advance, allowing AI systems to learn and improve over time.
   - The integration of

I now can give a great answer

Final Answer:
# The Evolution of Artificial Intelligence

**Introduction:**
Artificial Intelligence (AI) has rapidly transitioned from a distant vision of the future to a present-day reality that is reshaping industries and revolutionizing the way we live and work. In this blog post, we will explore the latest trends, prominent players, and significant news within the ever-evolving realm of AI.

## Latest Trends in Artificial Intelligence:

The field of AI continues to progress at a rapid pace, with machine learning algorithms leading the charge in empowering AI systems to learn and evolve based on data inputs. This progression has unlocked a myriad of possibilities across various sectors, particularly in the healthcare industry. AI is now being utilized for tasks such as diagnostics, personalized medicine, and enhancing patient care. Additionally, Natural Language Processing (NLP) technologies are enhancing customer service experiences by facilitating se

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Evolution of Artificial Intelligence

**Introduction:**
Artificial Intelligence (AI) has rapidly transitioned from a distant vision of the future to a present-day reality that is reshaping industries and revolutionizing the way we live and work. In this blog post, we will explore the latest trends, prominent players, and significant news within the ever-evolving realm of AI.

## Latest Trends in Artificial Intelligence:

The field of AI continues to progress at a rapid pace, with machine learning algorithms leading the charge in empowering AI systems to learn and evolve based on data inputs. This progression has unlocked a myriad of possibilities across various sectors, particularly in the healthcare industry. AI is now being utilized for tasks such as diagnostics, personalized medicine, and enhancing patient care. Additionally, Natural Language Processing (NLP) technologies are enhancing customer service experiences by facilitating seamless communication between humans and machines. AI-driven chatbots have gained traction for their ability to provide immediate customer support and optimize business operations.

## Key Players in the AI Industry:

Numerous key players are at the forefront of driving innovation within the AI landscape. Google's DeepMind has garnered attention for its pioneering work in deep learning and reinforcement learning, pushing the boundaries of AI research. IBM Watson has played a pivotal role in advancing AI applications across industries such as healthcare and finance. Amazon Web Services (AWS) offers a comprehensive suite of AI services, encompassing capabilities like image recognition, speech recognition, and text analysis. Microsoft Azure's Cognitive Services equip developers with intuitive tools to efficiently build AI applications.

## Noteworthy News in Artificial Intelligence:

Recent breakthroughs in AI have captivated the tech community and beyond. OpenAI's GPT-3 model has received acclaim for its capacity to generate human-like text and engage in natural language conversations. Tesla's strides in self-driving technology are reshaping the automotive sector, with plans for fully autonomous vehicles on the horizon. Facebook's AI research team is actively engaged in projects related to computer vision, natural language comprehension, and robotics, pushing the boundaries of AI capabilities.

**Target Audience:**
Our blog is tailored to professionals in the tech industry, business owners looking to implement AI solutions, and individuals keen on staying updated on the latest AI advancements. By delivering valuable insights and updates on AI trends, key industry players, and notable developments, we aim to empower our audience to make well-informed decisions within their respective domains.

**Call to Action:**
Stay abreast of the latest Artificial Intelligence developments and access exclusive content by subscribing to our newsletter for regular updates.

We are committed to providing a comprehensive overview of the AI landscape, offering valuable insights and information to our target audience.

Don't forget to subscribe to our newsletter for the latest updates on Artificial Intelligence!

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": "Liverpool F.C"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Liverpool F.C.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: Unveiling the Secrets of Liverpool F.C.: Latest Trends, Key Players, and Noteworthy News

Introduction:
- Brief overview of Liverpool F.C.'s history and achievements
- Setting the stage for discussing the latest trends, key players, and news

Key Points:
1. Latest Trends:
- Highlight the team's current performance in the Premier League and Champions League
- Discuss any recent transfers, injuries, or managerial changes
- Analyze fan engagement on social media platforms

2. Key Players:
-

I now can give a great answer

Final Answer:

# Unveiling the Secrets of Liverpool F.C.: Latest Trends, Key Players, and Noteworthy News

## Introduction
Liverpool F.C. boasts a rich history in the realm of football, with an extensive list of accomplishments that have cemented their position as one of the top clubs in the sport. In this exploration of the latest trends, key players, and noteworthy news surrounding the club, we aim to uncover the secrets that have propelled Liverpool F.C. to become a powerhouse in the Premier League and beyond.

## Latest Trends
Throughout the current season, Liverpool F.C. has been displaying remarkable performances in both the Premier League and the Champions League. With a formidable defensive lineup spearheaded by standout players such as Virgil van Dijk and a potent attacking force led by Mohamed Salah and Sadio Mane, the team has proven to be a formidable presence on the field. Recent transfers, injuries, and changes in management have added an el

In [15]:
Markdown(result)

# Unveiling the Secrets of Liverpool F.C.: Latest Trends, Key Players, and Noteworthy News

## Introduction
Liverpool F.C. boasts a rich history in the realm of football, with an extensive list of accomplishments that have cemented their position as one of the top clubs in the sport. In this exploration of the latest trends, key players, and noteworthy news surrounding the club, we aim to uncover the secrets that have propelled Liverpool F.C. to become a powerhouse in the Premier League and beyond.

## Latest Trends
Throughout the current season, Liverpool F.C. has been displaying remarkable performances in both the Premier League and the Champions League. With a formidable defensive lineup spearheaded by standout players such as Virgil van Dijk and a potent attacking force led by Mohamed Salah and Sadio Mane, the team has proven to be a formidable presence on the field. Recent transfers, injuries, and changes in management have added an element of suspense to the team's dynamics, ensuring that fans are kept on the edge of their seats. The club's social media platforms have been buzzing with fan engagement, showcasing the unwavering support of supporters from all corners of the globe.

## Key Players
Mohamed Salah, Virgil van Dijk, and Sadio Mane emerge as pivotal figures in Liverpool F.C.'s roster, each bringing their distinct skills and contributions to the team. Salah's lightning-fast speed and goal-scoring abilities, van Dijk's authoritative presence in defense, and Mane's versatility and ingenuity on the field have been fundamental to the club's success. The statistics and highlights of their recent performances serve as a testament to their influence on the team, illustrating why they are regarded as some of the best in the world.

## Noteworthy News
From significant milestones and events in the club's recent history to controversies and off-the-field developments, Liverpool F.C. has experienced it all. Quotes from players, managers, and fans add a personal touch to the club's narrative, offering insights into the emotions and fervor that envelop Liverpool F.C. both on and off the pitch. The club's illustrious history is brimming with triumphs and adversities, creating a captivating storyline for football aficionados and enthusiasts alike.

By delving into the latest trends, key players, and noteworthy news surrounding Liverpool F.C., we are able to gain a deeper comprehension of what sets the club apart. From their performances on the field to the passion that drives their fans, Liverpool F.C. continues to enthrall audiences worldwide with their distinctive brand of football. As we look towards the future, the enigmatic secrets of Liverpool F.C. are poised to unravel in ways that will undoubtedly keep us all eagerly anticipating what lies ahead.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).